In [ ]:
import numpy as np
from XS3VPU import XS3VPU

import warnings
warnings.filterwarnings(action='ignore')
import tensorflow as tf
print("Executing eagerly: {}".format(tf.executing_eagerly()))
warnings.filterwarnings(action='default')

vpu = XS3VPU(bpe=8)
int8min, int8max = vpu._sat_bounds(vpu._single)

In [ ]:
# enable training in notebook
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# Chunk

A chunk is an `acc_period` $\times$ `ve` size submatrix that operates on a single vector of data.

In [ ]:
def compute_chunk(vpu, W, x, W_start, W_step, x_start):
    # ~ 17 instructions
    vpu.VLDC(x[x_start:x_start+vpu.ve]); rw = W_start
    for _ in range(vpu.acc_period):  # unroll in asm
        vpu.VLMACCR(W[rw:rw+vpu.ve]); rw += W_step

In [ ]:
vpu = XS3VPU(bpe=8)

# generate coefficients and data for test
W = np.random.randint(int8min, int8max, size=(vpu.acc_period, vpu.ve), dtype=np.int8)
x = np.random.randint(int8min, int8max, size=(vpu.ve,), dtype=np.int8)

# reference output
y = np.matmul(np.int32(W), np.int32(x))

# mimic XS3 layout in memory
W_XS3 = np.copy(np.flipud(W).flatten())
x_XS3 = np.copy(x)

compute_chunk(vpu, W_XS3, x_XS3, W_start=0, W_step=vpu.ve, x_start=0)
print("Result matches reference: {}".format(np.all(np.all(vpu._combine_vD_vR() == y))))

# Tile

A tile is a sequence of chunks producing a single vector of outputs, while operating on continuously layed out data.

In [ ]:
def compute_tile(vpu, W, x, N_chunks,
                 W_start, W_step, W_chunk_step,
                 x_start, x_step):
    # ~ N_chunks * (17 + 2) + 5
    rx = x_start; rw = W_start
    for _ in range(N_chunks):
        compute_chunk(vpu, W, x, W_start=rw, W_step=W_step, x_start=rx)
        rx += x_step; rw += W_chunk_step

In [ ]:
vpu = XS3VPU(bpe=8)
N_chunks = 3
N = vpu.ve * N_chunks

# generate coefficients and data for test
W = np.random.randint(int8min, int8max, size=(vpu.acc_period, N), dtype=np.int8)
x = np.random.randint(int8min, int8max, size=(N,), dtype=np.int8)

# reference output
y = np.matmul(np.int32(W), np.int32(x))

# mimic XS3 layout in memory
W_XS3 = np.copy(np.flipud(W).flatten())
x_XS3 = np.copy(x)

compute_tile(vpu, W_XS3, x_XS3, N_chunks=N_chunks,
             W_start=0, W_step=N, W_chunk_step=vpu.ve,
             x_start=0, x_step=vpu.ve)
print("Result matches reference: {}".format(np.all(np.all(vpu._combine_vD_vR() == y))))

# Band

A band is a sequence of tiles producing a single vector of outputs, operating on a sequence of equally spaced continously layed out data vectors.

In [ ]:
def compute_band(vpu, W, x, N_chunks, N_tiles,
                 W_start, W_step, W_chunk_step, W_tile_step,
                 x_start, x_step, x_tile_step):
    # N_tiles * (N_chunks * (17 + 2) + 5 + 2) + 5
    rx = x_start; rw = W_start
    for _ in range(N_tiles):
        compute_tile(vpu, W, x, N_chunks,
                     W_start=rw, W_step=W_step, W_chunk_step=W_chunk_step,
                     x_start=rx, x_step=x_step)
        rx += x_tile_step; rw += W_tile_step

In [ ]:
vpu = XS3VPU(bpe=8)
N_chunks, N_tiles = 3, 2
N = vpu.ve * N_chunks * N_tiles

# generate coefficients and data for test
W = np.random.randint(int8min, int8max, size=(vpu.acc_period, N), dtype=np.int8)
x = np.random.randint(int8min, int8max, size=(N,), dtype=np.int8)

# reference output
y = np.matmul(np.int32(W), np.int32(x))

# mimic XS3 layout in memory
W_XS3 = np.copy(np.flipud(W).flatten())
x_XS3 = np.copy(x)

compute_band(vpu, W_XS3, x_XS3, N_chunks=N_chunks, N_tiles=N_tiles,
             W_start=0, W_step=N, W_chunk_step=vpu.ve, W_tile_step=N_chunks*vpu.ve,
             x_start=0, x_step=vpu.ve, x_tile_step=N_chunks*vpu.ve)
print("Result matches reference: {}".format(np.all(np.all(vpu._combine_vD_vR() == y))))

# Dense matrix multiplication

In [ ]:
def XS3_matmul(vpu, W, x, y, N_bands, N_chunks):
    # ~ N_bands * (N_chunks * (17 + 2) + 5 + 8) + 5
    rw = 0; ry = 0
    for _ in range(N_bands):
        vpu.VCLRDR()  # TODO add bias loading
        compute_tile(vpu, W, x, N_chunks,
                     W_start=rw, W_step=N_chunks*vpu.ve, W_chunk_step=vpu.ve,
                     x_start=0, x_step=vpu.ve)
        y[ry:ry+vpu.acc_period] = vpu._combine_vD_vR()  # VLSAT, VPOS, VSTRPV
        rw += vpu.acc_period * N_chunks * vpu.ve; ry += vpu.acc_period

In [ ]:
vpu = XS3VPU(bpe=8)
N_chunks, N_bands = 3, 2
N = N_chunks * vpu.ve
M = N_bands * vpu.acc_period

# generate coefficients and data
W = np.random.randint(int8min, int8max, size=(M, N), dtype=np.int8)
x = np.random.randint(int8min, int8max, size=(N,), dtype=np.int8)

# reference output
y = np.matmul(np.int32(W), np.int32(x))

# mimic XS3 layout in memory
W_XS3 = [np.flipud(W[j:j+vpu.acc_period, :]) for j in range(0, W.shape[0], vpu.acc_period)]
W_XS3 = np.copy(np.vstack(W_XS3).flatten())
x_XS3 = np.copy(x)
y_XS3 = np.zeros(y.shape, dtype=y.dtype)

XS3_matmul(vpu, W_XS3, x_XS3, y_XS3, N_bands, N_chunks)
print("Output matches reference: {}".format(np.all(y_XS3 == y)))

# 1x1 Convolution

In [ ]:
def XS3_conv2d_1x1(vpu, K, x, y, height, width, C_out_bands, C_in_chunks):
    rx = 0; ry = 0;
    for _ in range(height * width):
        XS3_matmul(vpu, K, x[rx:], y[ry:], C_out_bands, C_in_chunks)
        rx += C_in_chunks * vpu.ve
        ry += C_out_bands * vpu.acc_period

In [ ]:
vpu = XS3VPU(bpe=8)
C_in_chunks, C_out_bands = 2, 3
C_in = C_in_chunks * vpu.ve
C_out = C_out_bands * vpu.acc_period
width = 4
height = 6

# generate kernels and data
K = np.random.randint(int8min, int8max, size=(C_out, C_in), dtype=np.int8)
D = np.random.randint(int8min, int8max, size=(height, width, C_in), dtype=np.int8)

# convert to tf.float64, because tf.nn.conv2d cannot handle integer tensors
D_tf = tf.convert_to_tensor(D, dtype=tf.float64)
D_tf = tf.expand_dims(D_tf, axis=0)
K_tf = tf.convert_to_tensor(K.T, dtype=tf.float64)
K_tf = tf.expand_dims(tf.expand_dims(K_tf, axis=0), axis=0)

# reference output
Y_tf = tf.nn.conv2d(D_tf, K_tf, strides=1, padding="VALID", data_format='NHWC')
Y_tf = tf.cast(Y_tf, tf.int32)
Y = Y_tf.numpy()

# mimic XS3 layout in memory
K_XS3 = [np.flipud(K[j:j+vpu.acc_period, :]) for j in range(0, K.shape[0], vpu.acc_period)]
K_XS3 = np.copy(np.vstack(K_XS3).flatten())
D_XS3 = np.copy(D.flatten())
Y_XS3 = np.zeros((height, width, C_out), dtype=np.int32).flatten()

XS3_conv2d_1x1(vpu, K_XS3, D_XS3, Y_XS3, height, width, C_out_bands, C_in_chunks)
print("Output matches reference: {}".format(np.all(Y_XS3 == Y.flatten())))

# NxK Convolution

In [ ]:
def XS3_conv2d(vpu, K, x, y, height, width, K_h, K_w, C_out_bands, C_in_chunks):
    # (height - K_h + 1) * (width - K_w + 1) * (C_out_bands * (K_h * (K_w*C_in_chunks * (17 + 2) + 5 + 2) + 5 + 8) + 5)
    rx = 0;
    for hi in range(height - K_h + 1):
        for wi in range(width - K_w + 1):
            rw = 0;
            rx = (hi * width + wi) * C_in_chunks * vpu.ve
            ry = (hi * (width - K_w + 1) + wi) * C_out_bands * vpu.acc_period
            for _ in range(C_out_bands):
                vpu.VCLRDR()  # load bias
                compute_band(vpu, K, x, N_chunks=K_w*C_in_chunks, N_tiles=K_h,
                             W_start=rw, W_step=K_h*K_w*C_in_chunks*vpu.ve,
                             W_chunk_step=vpu.ve, W_tile_step=K_w*C_in_chunks*vpu.ve,
                             x_start=rx, x_step=vpu.ve, x_tile_step=width*C_in_chunks*vpu.ve)
                y[ry:ry+vpu.acc_period] = vpu._combine_vD_vR()  # VLSAT, VPOS, VSTRPV
                rw += vpu.acc_period * K_h * K_w * C_in_chunks * vpu.ve;
                ry += vpu.acc_period

In [ ]:
vpu = XS3VPU(bpe=8)
C_in_chunks, C_out_bands = 2, 3
C_in = C_in_chunks * vpu.ve
C_out = C_out_bands * vpu.acc_period
height, width = 7, 5
K_h, K_w = 3, 4

# generate kernels and data
K = np.random.randint(int8min, int8max, size=(C_out, K_h, K_w, C_in), dtype=np.int8)
D = np.random.randint(int8min, int8max, size=(height, width, C_in), dtype=np.int8)

# convert to tf.float64, because tf.nn.conv2d cannot handle integer tensors
D_tf = tf.convert_to_tensor(D, dtype=tf.float64)
D_tf = tf.expand_dims(D_tf, axis=0)
K_tf = tf.convert_to_tensor(K, dtype=tf.float64)
K_tf = tf.transpose(K_tf, perm=[1, 2, 3, 0])

# reference output
Y_tf = tf.nn.conv2d(D_tf, K_tf, strides=1, padding="VALID", data_format='NHWC')
Y_tf = tf.cast(Y_tf, tf.int32)
Y = Y_tf.numpy()

# mimic XS3 layout in memory
K_XS3 = [np.flip(K[j:j+vpu.acc_period, :, :, :], axis=0) for j in range(0, K.shape[0], vpu.acc_period)]
K_XS3 = np.copy(np.vstack(K_XS3).flatten())
D_XS3 = np.copy(D.flatten())
Y_XS3 = np.zeros((height - K_h + 1, width - K_w + 1, C_out), dtype=np.int32).flatten()

XS3_conv2d(vpu, K_XS3, D_XS3, Y_XS3, height, width, K_h, K_w, C_out_bands, C_in_chunks)
print("Output matches reference: {}".format(np.all(Y_XS3 == Y.flatten())))

# 3 channel input layer convolution

In [ ]:
def XS3_conv2d_input(vpu, K, x, y, height, width, K_h, K_w, C_out_bands):
    # (height - K_h + 1) * (width - K_w + 1) * (C_out_bands * (K_h * ((17 + 2) + 5 + 2) + 5 + 8) + 5)
    rx = 0;
    for hi in range(height - K_h + 1):
        for wi in range(width - K_w + 1):
            rw = 0;
            rx = (hi * width + wi) * 4
            ry = (hi * (width - K_w + 1) + wi) * C_out_bands * vpu.acc_period
            for _ in range(C_out_bands):
                vpu.VCLRDR()  # load bias
                compute_band(vpu, K, x, N_chunks=1, N_tiles=K_h,
                             W_start=rw, W_step=K_h*vpu.ve, W_chunk_step=vpu.ve, W_tile_step=vpu.ve,
                             x_start=rx, x_step=4, x_tile_step=width*4)
                y[ry:ry+vpu.acc_period] = vpu._combine_vD_vR()  # VLSAT, VPOS, VSTRPV
                rw += vpu.acc_period * K_h * vpu.ve;
                ry += vpu.acc_period

In [ ]:
vpu = XS3VPU(bpe=8)
C_out_bands = 2
C_in = 3  # works for C_in <= 4
C_out = C_out_bands * vpu.acc_period
height, width = 15, 9
K_h, K_w = 11, 8  # works for K_w <= 8

# generate kernels and data
K = np.random.randint(int8min, int8max, size=(C_out, K_h, K_w, C_in), dtype=np.int8)
D = np.random.randint(int8min, int8max, size=(height, width, C_in), dtype=np.int8)

# convert to tf.float64, because tf.nn.conv2d cannot handle integer tensors
D_tf = tf.convert_to_tensor(D, dtype=tf.float64)
D_tf = tf.expand_dims(D_tf, axis=0)
K_tf = tf.convert_to_tensor(K, dtype=tf.float64)
K_tf = tf.transpose(K_tf, perm=[1, 2, 3, 0])

# reference output
Y_tf = tf.nn.conv2d(D_tf, K_tf, strides=1, padding="VALID", data_format='NHWC')
Y_tf = tf.cast(Y_tf, tf.int32)
Y = Y_tf.numpy()

# mimic XS3 layout in memory
K_XS3 = [np.flip(K[j:j+vpu.acc_period, :, :, :], axis=0) for j in range(0, K.shape[0], vpu.acc_period)]
K_XS3 = np.concatenate(K_XS3, axis=0)
K_XS3 = np.pad(K_XS3, pad_width=[(0,0), (0,0), (0,8-K_w), (0,4-C_in)])
K_XS3 = np.copy(K_XS3.flatten())

D_XS3 = np.concatenate([D, np.zeros(shape=(height, width, 4-C_in), dtype=np.int8)], axis=2)
D_XS3 = np.copy(D_XS3.flatten())
D_XS3 = np.pad(D_XS3, pad_width=((0, (8-K_w)*4),))

Y_XS3 = np.zeros((height - K_h + 1, width - K_w + 1, C_out), dtype=np.int32).flatten()

XS3_conv2d_input(vpu, K_XS3, D_XS3, Y_XS3, height, width, K_h, K_w, C_out_bands)
print("Output matches reference: {}".format(np.all(Y_XS3 == Y.flatten())))

# Estimations

Throughout the following estimates we assume a single tile (sic!) of Phoenix. Scaling neural network inference speed for parallelization across tiles in not trivial. In some cases a 2x speedup may be possible for two tiles, but in most cases, this factor would be smaller.

In [ ]:
instr_buffer_factor = 5/4  # estimated overhead of fetching instructions, <= 5/4
CLOCK_216, CLOCK_600, CLOCK_800 = 216e6, 600e6, 800e6

In [ ]:
from math import ceil

def max_pool_cnt(height, width, depth, price_per_vector=8):
    # an element-wise max on two vectors can be calculated using 8 insturctions
    # the fudge factor is estimated by doing this three times for each 2x2 max pool kernel
    return (width // 2) * (height // 2) * (depth // 32) * 3 * price_per_vector

def conv2d_cnt(height, width, C_in, C_out, K_h=5, K_w=5):
    num_out_pixels = height * width
    return num_out_pixels * ((C_out//16) * (K_h * (K_w*(C_in//32) * (17 + 2) + 5 + 2) + 5 + 8) + 5)

def conv2d_cnt_input(height, width, C_out, K_h=5):
    num_out_pixels = height * width
    return num_out_pixels * ((C_out//16) * (K_h * ((17 + 2) + 5 + 2) + 5 + 8) + 5)

def fc_cnt(N_in, N_out):
    return (N_out // 16) * ((N_in // 32) * (17 + 2) + 5 + 8) + 5

def max_pool_cnt_single_layer(height, width, price_per_vector=8):
    # do a 2x1 max pool first
    # then a 1x2 max pool using a trick by switching to 16 bit mode and shifting down by 8bits (extra 5 instructions)
    return (height // 2) * (ceil(width/32) * price_per_vector + width / 32 * (price_per_vector + 5))

def ave_pool_cnt_single_layer(height, width, price_per_vector=16):
    # roughly 16 instruction for 2x16 8-bit vectors
    return (height / 2) * (width / 16) * price_per_vector

### ARM CMSIS-NN

Estimate our performance on the exact network described here:

https://community.arm.com/developer/ip-products/processors/b/processors-ip-blog/posts/new-neural-network-kernels-boost-efficiency-in-microcontrollers-by-5x

Note that the below estimates holds for a single thread executing the layers sequentially. We estimate that since the bulk of the computation can be spatially parallelized within each conv2d layer, we could achieve a 5x performance increase on a single tile.

In this benchmark, the M7 was running at 216MHz. Using a 28nm process, there are claims that their clock rate can reach 800MHz (https://www.theregister.co.uk/2014/09/24/arm_cortex_m7/). In practice, it seems that the highest this goes is 600 MHz (https://www.nxp.com/products/processors-and-microcontrollers/arm-microcontrollers/i.mx-rt-series:IMX-RT-SERIES). Thus, the reported benchmarks for the M7 are going to be scaled for a 600 MHz clock speed.

In [ ]:
cnt = []
# layer 1, specialized implementation for C_in <= 4 and K_w <= 8
cnt.append(conv2d_cnt_input(height=32, width=32, C_out=32, K_h=5))
# layer 2
cnt.append(max_pool_cnt(height=32, width=32, depth=32))
# layer 3
cnt.append(conv2d_cnt(height=16, width=16, C_in=32, C_out=32))
# layer 4
cnt.append(max_pool_cnt(height=16, width=16, depth=32))
# layer 5
cnt.append(conv2d_cnt(height=8, width=8, C_in=32, C_out=64))
# layer 6
cnt.append(max_pool_cnt(height=8, width=8, depth=64))
# layer 7, scale down number for 16 output proportionally to 10 outputs
cnt.append(fc_cnt(N_in=4*4*64, N_out=16) * 10/16)

t_est_eff = np.sum(cnt) * instr_buffer_factor * 5 / CLOCK_600
print("Estimate with efficient input and output layers @600MHz: {:.2f} ms".format(t_est_eff * 1e3))
print("Layer-wise break down of instruction counts:")
for j, c in enumerate(cnt):
    print("Layer {}: {:.0f}".format(j+1, c))

In [ ]:
t_est_eff_par = t_est_eff / 5
t_arm = 99.1e-3 / CLOCK_600 * CLOCK_216
print("Summary of inference on the network above with Pheonix @600MHz:")
print("Parallelized inference would take ~{:.2f} ms".format(t_est_eff_par * 1e3))
print("This is a ~{:d}x speedup compared to ARM's benchmark of {:.1f} ms".format(
      int(t_arm / t_est_eff_par), t_arm * 1e3))
print()

t_est_eff_par_800 = t_est_eff_par / CLOCK_800 * CLOCK_600
print("Summary of inference on the network above with Pheonix @800MHz:")
print("Parallelized inference would take ~{:.2f} ms".format(t_est_eff_par_800 * 1e3))
print("Phoenix @800MHz delivers a ~{:d}x speedup compared to M7 @600MHz".format(int(t_arm/t_est_eff_par_800)))

### GreenWaves GAP8

Estimate our performance on the layers described here:

https://greenwaves-technologies.com/gap8-cnn-benchmarks/

The GAP8 chip parallelizes convolutions differently: it evaluates each input layer sequentially, which is not the most efficient use of our vector unit. These calculations are less reliable, but were done to intentionally overestimate our instruction counts. Our system would also scale much better as the number of input and output layers grow. Our approach also tends to be agnostic of the image and kernel size, while GAP8's approach has hard assumptions on possible kernel sizes, outside which it would quickly lose performance.

GAP8 uses a 55nm process, on which we could do 600MHz. In all the following benchmarks we will assume this clockrate for Phoenix. We also assume a 5x speedup for parallelized implementations of Phoenix. The benchmarks for the GAP8 chip are taken from their fastest ("parallel vector") implementation, which cannot be scaled further without increasing a clock rate. Their reported clock speed is 175MHz, which is the maximum for their architecture.

In [ ]:
m_cnt = max_pool_cnt_single_layer(height=112, width=112)
print("Estimate of 2x2/2 max pooling on a 112x112 input: {:.2f} us".format(
      m_cnt * instr_buffer_factor * 5 / CLOCK_600 * 1e6))
print("Estimate when parallelized: {:.2f} us".format(
      m_cnt * instr_buffer_factor * 5 / CLOCK_600 * 1e6 / 5))
print("GAP8's benchmark is: {} us".format(2014/100))

In [ ]:
a_cnt = ave_pool_cnt_single_layer(height=112, width=112)
print("Estimate of 2x2/2 average pooling on a 112x112 input: {:.2f} us".format(
      a_cnt * instr_buffer_factor * 5 / CLOCK_600 * 1e6))
print("Estimate when parallelized: {:.2f} us".format(
      a_cnt * instr_buffer_factor * 5 / CLOCK_600 * 1e6 / 5))
print("GAP8's benchmark is: {} us".format(1861/100))

In [ ]:
# we scale down farely well for fewer output channels, so apply a simple scaling
c_cnt = conv2d_cnt_input(height=(112-5+1), width=(112-5+1), C_out=16, K_h=5) / 16
print("Estimate of 5x5 conv2d on a 112x112 input: {:.2f} us".format(
      c_cnt * instr_buffer_factor * 5 / CLOCK_600 * 1e6))
print("Estimate when parallelized: {:.2f} us".format(
      c_cnt * instr_buffer_factor * 5 / CLOCK_600 * 1e6 / 5))
print("GAP8's benchmark is: {} us".format(12848/100))

In [ ]:
# our hypothesis is that the most efficient implementation for 16 columns
# would be to use zero padding on the weights, and calculate 32 columns
f_cnt = fc_cnt(N_in=32, N_out=1024)
print("Estimate of 1024x16 fully connected layer : {:.2f} us".format(
      f_cnt * instr_buffer_factor * 5 / CLOCK_600 * 1e6))
print("Estimate when parallelized: {:.2f} us".format(
      f_cnt * instr_buffer_factor * 5 / CLOCK_600 * 1e6 / 5))
print("GAP8's benchmark is: {} us".format(1252/100))

Suggestion: instead of using the same benchmark as GAP8, we should report the number of 5x5 single layer convolutions our chip can perform per second, and calculate this at the most optimal setup for our chip (Undoubtedly, GAP8 chose this metric because they have significant overhead as the number of layers grow).

In [ ]:
batch_cnt = conv2d_cnt(height=(112-5+1), width=(112-5+1), C_in=32, C_out=16)
batch_time = batch_cnt * instr_buffer_factor * 5 / CLOCK_600
print("Estimated number of 5x5 conv2d on a 112x112 image @600MHz: {:.2f}/s".format(
      (32*16) / batch_time))
print("Estimate when parallelized: {:.2f}/s".format(
      5 * (32*16) / batch_time))
print("GAP8's benchmark (without overhead) is: {:.2f}/s".format(1 / (12848/100 / 1e6)))
print()

print("Summary: (Assuming a very reasonable 20% overhead for GAP8)")
print("We have a ~{:.2f}x speedup compared to GAP8's benchmark in terms of convolution operations per second.".format(
    5 * (32*16) / batch_time * (1.2 * (12848/100) / 1e6)))